In [ ]:
import os
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

# Load API key
HUGGINGFACEHUB_API_KEY = os.getenv("HUGGINGFACEHUB_API_TOKEN")

# Example text
text = """
LangChain is a framework for developing applications powered by large language models (LLMs).
It helps with chaining together prompts, models, and data sources."""

# Split text
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents([text])

# Create embeddings & vector DB
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(docs, embeddings)

# Load LLM
llm = HuggingFaceHub(
    repo_id="google/flan-t5-base",
    huggingfacehub_api_token=HUGGINGFACEHUB_API_KEY
)

# Build RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=docsearch.as_retriever(),
    return_source_documents=True
)

# Streamlit UI
st.header("Simple Q/A Bot")

user_question = st.text_input("Ask a question:")

if user_question:
    result = qa({"query": user_question})
    st.write("Answer:")
    st.write(result["result"])

    with st.sidebar:
        st.subheader("Retrieved Context")
        for doc in result["source_documents"]:
            st.write(doc.page_content[:300])
